# Workshop_02  2016-10-21
The reference structure I choose to reproduce is:
![alt text](http://www.ragusanews.com/immagini_articoli/24-02-2014/1396117860-aaa-cemento-armato-su-tre-piani-in-zona-agricola-vista-paesaggio.jpg "Riferimento")

Delaying the problem of the beams between the pillars frames (*i telai piani*), the first solution that has come to my mind is to evaluate every line of the CVS file in order.
So, the subproblems are:
1. Split every line of the CVS in a list of lists
2. Generate a PyPlasm object (either an HPC of pillar frames or a translation) from the list created in step 1
3. Generate a list of PyPlasm objects ready for the STRUCT function

In [1]:
#I run the code of the workshop_01. It imports larlib and also provides the method for generating the pillar frames
%run frameGenerator.py

Evaluating fenvs.py..
...fenvs.py imported in 0.02 seconds


Here the first solution proposed:

In [4]:
from larlib import *
import frameGenerator

def strToParameters(string):
    """Creates a list of lists of floats from a CSV line"""
    cells = string.split(';')
    final = []
    for element in cells:
        final.append([float(num) for num in element.split(',')])
    return final
    
def generateListElement(param):
    """Given a list, it returns a list of pyPlasm objects"""
    
    if len(param) == 1:
        return [T(i+1)(param[0][i]) for i in range(len(param[0]))]
    else:
        return [frameGenerator.createStructure(param[0],param[1],param[2],param[3],'pillars>beams')]
        
def stringsToCG(lines):
    """Given a list of strings returns a list of pyPlasm functions"""
    
    ret = []
    for line in lines:
        ret.extend(generateListElement(strToParameters(line)))
    return ret
    
def ggpl_bone_structure(file_name):
    with open(file_name) as csv:
        return STRUCT(stringsToCG(csv.readlines()))

This solutions works. Let's test it with a CVS representing the reference structure.

In [5]:
ggpl_bone_structure('frame_data_486815.csv')

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x6cae720> >

Ok, it generates an Hpc. Let's visualize it.

In [6]:
VIEW(ggpl_bone_structure('frame_data_486815.csv'))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x6cae6c0> >

![alt text](./images/frame1.png "Riferimento")
Ops. Pillar frames are pretty near. Now I will use a new CSV:

Ops. Pillar frames are pretty near. Now I will use a new CSV:

In [9]:
VIEW(ggpl_bone_structure('frame_data_486815_v2.csv'))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x6cae630> >

Ok, looks good:
![alt text](./images/frame2.png "Riferimento")
But pillars and beams seems too thin. Here another test:

In [11]:
VIEW(ggpl_bone_structure('frame_data_486815_v3.csv'))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x6cae510> >

Ok, better.
![alt text](./images/frame3.png "Frame 3")
